# A3: A\*, IDS, and Effective Branching Factor

Michael Lynn

For this assignment, implement the Recursive Best-First Search
implementation of the A\* algorithm given in class.  Name this function `aStarSearch`.  Also in this notebook include your `iterativeDeepeningSearch` functions.  Define a new function named `ebf` that returns an estimate of the effective
branching factor for a search algorithm applied to a search problem.

So, the required functions are

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth, precision=0.01)`, returns number of nodes expanded and depth reached during a search.

Apply `iterativeDeepeningSearch` and `aStarSearch` to several eight-tile sliding puzzle
problems. For this you must include your implementations of these functions, from Assignment 2:

  * `actionsF_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. With each action include a step cost of 1. For example, if all four actions are possible from this state, return [('left', 1), ('right', 1), ('up', 1), ('down', 1)].
  * `takeActionF_8p(state, action)`: return the state that results from applying `action` in `state` and the cost of the one step,
  
plus the following function for the eight-tile puzzle:

  * `goalTestF_8p(state, goal)`
  
Compare their results by displayng
solution path depth, number of nodes 
generated, and the effective branching factor, and discuss the results.  Do this by defining the following function that prints the table as shown in the example below.

   - runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3])
   
Define this function so it takes any number of $h$ functions in the list that is the fourth argument.

## Heuristic Functions

For `aStarSearch` use the following two heuristic functions, plus one more of your own design, for a total of three heuristic functions.

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = ?$, that you define.  It must be admissible, and not constant for all states.

## Comparison

Apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

Print a well-formatted table like the following.  Try to match this
format. 

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Of course you will have one more line for `h3`.

# Methods
Methods implemented were starsearch, iterativeDeepeningSearch, runexperiement and ebf.
The starsearch was ran against h1,h2 and h3 heuristics functions, the ebf is used to test how well the 
heuristics are with a percision of 0.01.

# Obervations
I compared the heuristcs methods to my iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth) function. The test include the number of nodes expanded, how far the expansion extended and the effective branching factor

In [252]:
import math


def printState_8p(startState):
    #print("printing: ",startState)
    newList = startState.copy()
    for i, myValue in enumerate(startState):
        if myValue == 0:
            #print("Found empty space defined by a", myValue)
            newList[i] = "-"


    print(" ".join(map(str,newList[0:3])))
    print(" ".join(map(str,newList[3:6])))
    print(" ".join(map(str,newList[6:9])))


def actionsF_8p(startState):
    # get coord of my current position
    myEmptyCoord = findBlank_8p(startState)
    yPos = myEmptyCoord[0]
    xPos = myEmptyCoord[1]
    myMoves = []

    # possible moves from current positionn
    if xPos > 0:
        item = "left",1
        myMoves.append(item)

    if xPos < 2:
        item = "right",1
        myMoves.append(item)

    if yPos > 0:
        item = "up",1
        myMoves.append(item)

    if yPos < 2:
        item = "down",1
        myMoves.append(item)

    return myMoves

def findBlank_8p(startState):

     # get coord of my current position
    xPos = startState.index(0)
    yPos = 0

    #  row/ col position only works for a 3x3(hard coded)
    if xPos >= 0 and xPos <= 2:
        #print("coorindartes of the empty space are (y,x)", 0, xPos)
        return (yPos, xPos)

    if xPos >= 3 and xPos <= 5:
        rowOffset = xPos - 3
        #print("coorindartes of the empty space are (y,x)",1, rowOffset)
        return (yPos + 1, rowOffset)

    if xPos >= 6 and xPos <= 8:
        rowOffset = xPos - 6
        #print("coorindartes of the empty space are (y,x)",2, rowOffset)
        return (yPos + 2, rowOffset)



def takeActionF_8p(startState, action):
    import copy
    myState = startState.copy()
    rowX, colY = findBlank_8p(myState)
    tempRow = 0
    tempCol = 0
    
    if action[0] is 'left':
        tempCol -= 1
    elif action[0] is 'right':
        tempCol += 1
    elif action[0] is 'up':
        tempRow -= 1
    elif action[0] is 'down':
        tempRow += 1
       
    offsetY, offsetC = rowX + tempRow, colY + tempCol
    offset = offsetY * 3 + offsetC
    offset2 = rowX  * 3 + colY
    myState[offset2] = myState[offset]
    offset3 = offsetY  * 3 + offsetC
    myState[offset3] = 0
    return (myState, 1)



def printPath_8p(startState, goalState, myPath):
    printState_8p(startState)
    print("to")
    printState_8p(goalState)
    print("is {} nodes long".format(len(myPath)))
    print()
    for myMoves in myPath:
      printState_8p(myMoves)
      print()


def depthLimitedSearch(startState, goalState,
                       actionsF, takeActionF, moveLimit):
    global itsTotal
    if startState == goalState:
        #print("baseCase hit")
        #printState_8p(goalState)
        #print("-----------------------------")
        return []
    if moveLimit == 0:
        #print("moveLimit hit")
        return "cutoff"

    limitReached = False
    for myMove in actionsF(startState):
        itsTotal += 1
        myNewState = takeActionF(startState, myMove)[0]
        myResult = depthLimitedSearch(myNewState, goalState,
                                    actionsF, takeActionF, moveLimit-1)
        if myResult is 'cutoff':
            limitReached = True
        elif myResult is not 'failure':
            myResult.insert(0, myNewState)
            return myResult
    if limitReached:
        return 'cutoff'
    else:
        return 'failure'




def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, myDepth):
    #set range of recusive searching and pass it as a depthLimitedSearch parameter
    global itsTotal
    global itsDepth
    itsTotal = 0
    itsDepth = 0

    for moveLimit in range(myDepth):
        myResult = depthLimitedSearch(startState, goalState, actionsF, takeActionF, moveLimit)

        #check if base case is returned
        if myResult is not 'cutoff':
            #print("base case hit")
            myResult.insert(0, startState)
            itsDepth = moveLimit
            return myResult

    return "cutoff"

def ebf(nNodes, depth, precision = 0.01):
    if nNodes == depth:
        return 0
    if depth == 0:
        return 1.0

    k = 2.0
    b = depth/k

    myMin = 0.0
    myMax = nNodes
    myTotal = 0

    myMinRange = nNodes - (nNodes*precision)
    myMaxRange = nNodes + (nNodes*precision)

    while ( not(myTotal > myMinRange and myTotal < myMaxRange)):
        myTotal = ( (1.0-(b**(depth + 1) ) ) / (1.0 - b) )

        if myTotal < nNodes:
            myMin = b
            b = (b + myMax) / k

        if myTotal > nNodes:
            myMax = b

        b = (b + myMin) / k

    return b

def goalTestF_8p(state, goalState):
    return state == goalState

def goalTestF_simple(state, goalState):
    return state == goalState

def h1_8p(state, goal):
    return 1

class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

def aStarSearch(startState, actionsF, takeActionF, goalTestF, Fx):
    aStarDepth = 0
    aStarCounter = 0
    
    h = Fx(startState)
    myNodeBegin = Node(state=startState, f = 0+h, g=0, h=h)
    
    return StarSearch(myNodeBegin, actionsF, takeActionF, goalTestF, Fx, float('inf'))

def StarSearch(myParent, actionsF, takeActionF, goalTestF, Fx, myMax):
    aStarDepth = 0
    aStarCounter = 0
    
    # base case
    if goalTestF(myParent.state):
        return ([myParent.state], myParent.g)
   
    actions = actionsF(myParent.state)
    #f ailed base case
    if not actions:
        return ("failure", float('inf'))
    
    
    
    myChild = []   
    for action in actions:
        aStarCounter += 1
        (childState, stepCost) = takeActionF(myParent.state, action)
        
        h = Fx(childState)
        g = myParent.g + stepCost
        f = max(h+g, myParent.f)
         
        # add actions temp to mylist
        temp = Node(state=childState, f=f, g=g, h=h)
        myChild.append(temp)
        
        
    while True: 
        myChild.sort(key = lambda n: n.f)
        
        firstBest = myChild[0]
        
        if firstBest.f > myMax:
            return ("failure",firstBest.f)

        secondBest = myChild[1].f if len(myChild) > 1 else float('inf')
        
        #recursive search
        result,firstBest.f = StarSearch(firstBest, actionsF, takeActionF, goalTestF,
                                            Fx, min(myMax,secondBest))
        if result is not "failure":
            aStarDepth += 1
            
            result.insert(0,myParent.state)
            
            return (result, firstBest.f)

def coloffset(colx, coly):

    if colx == 0 or colx == 3 or colx == 6:
        xy = 0
    elif colx == 1 or colx == 4 or colx == 7:
        xy = 1
    else:
        xy = 2
    # check offset col
    if coly == 0 or coly == 3 or coly == 6:
        yx = 0
    elif coly == 1 or coly == 4 or coly == 7:
        yx = 1
    else:
        yx = 2

    return math.fabs(yx - xy)


def rowOffset(rowx, rowy):
    if rowx == 0 or rowx == 1 or rowx == 2:
        xy = 0
    elif rowx == 3 or rowx == 4 or rowx == 5:
        xy = 1
    else:
        xy = 2
    #check offset row
    if rowy == 0 or rowy == 1 or rowy == 2:
        yx = 0
    elif rowy == 3 or rowy == 4 or rowy == 5:
        yx = 1
    else:
        yx = 2
   
    return math.fabs(yx - xy)



def h1_8p(startState, goalState):
    return 1

def h3_8p(startState, goalState):
    k = goalState[0] / 2
    return math.fabs(startState[0] - k)

def h2_8p(startState, goalState):
    yPos = coloffset(startState[0],  goalState[0])
    xPos = rowOffset(startState[0],  goalState[0])
    return yPos + xPos

def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goalState):
    return state == goalState

def h_simple(state, goal):
    return 1

def runExperiment(goalState1, goalState2, goalState3, heuristics = []):
    global aStarDepth
    global aStarCounter
    aStarDepth = 0
    aStarCounter = 0
    startState = [1, 0, 3, 4, 2, 5, 6, 7, 8]
    print("{!s}       {!s}     {!s}".format(goalState1, goalState2, goalState3))
    
    print("Algorithm      Depth     Nodes   EBF           Depth   Nodes   EBF               Depth     Nodes        EBF")

    k = 999999
    k2 = 0.01
    iterativeDeepeningSearch(startState, goalState1, actionsF_8p, takeActionF_8p, k)
    
    print("IDS 			 {!s}       {!s}     {!s}".format(itsDepth, itsTotal, round(ebf(itsTotal, itsDepth, k2), 3)), end='')

    iterativeDeepeningSearch(startState, goalState2, actionsF_8p, takeActionF_8p, k)
    print("            {!s}      {!s}    {!s}".format(itsDepth, itsTotal, round(ebf(itsTotal, itsDepth, k2), 3)), end='')
    

    iterativeDeepeningSearch(startState, goalState3, actionsF_8p, takeActionF_8p, k)
    
    print("                  {!s}    {!s}      {!s}".format(itsDepth, itsTotal, round(ebf(itsTotal, itsDepth, k2), 3)))
    tempC = 1
    
    
    
    for H in heuristics:
        aStarSearch(startState, actionsF_8p, takeActionF_8p, lambda go: goalTestF_simple(go, goalState1), lambda go: H(go, goalState1))

        eb = round(ebf(aStarCounter, aStarDepth, k2),3)
        print("{}       		 {!s}      {!s}     {!s}".format(tempC, aStarDepth, aStarCounter,eb), end='')

        aStarSearch(startState, actionsF_8p, takeActionF_8p, lambda go: goalTestF_simple(go, goalState2), lambda go: H(go, goalState2))
        
        print("{}       		 {!s}      {!s}     {!s}".format(tempC, aStarDepth, aStarCounter,eb), end='')

        aStarSearch(startState, actionsF_8p, takeActionF_8p, lambda go: goalTestF_simple(go, goalState3), lambda go: H(go, goalState3))
        
        print("{}       		 {!s}      {!s}     {!s}".format(tempC, aStarDepth, aStarCounter,eb))
        
        tempC += 1

First, some example output for the ebf function.

In [253]:
ebf(10, 3)

1.6494140625

The smallest argument values should be a depth of 0, and 1 node.

In [254]:
ebf(1, 0)

1.0

In [255]:
ebf(2, 1)

0.9453125

In [256]:
ebf(2, 1, precision=0.000001)

1.0000027250498533

In [257]:
ebf(200000, 5)

12.131990194320679

In [258]:
ebf(200000, 50)

1.2349419295787811

Here is a simple example using our usual simple graph search.

In [259]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [260]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [261]:
takeActionF_simple('a', actions[0])

('b', 1)

In [262]:
goalTestF_simple('a', 'a')

True

In [263]:
h_simple('a', 'z')

1

In [264]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [265]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

NOT AVAILABLE YET!  Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A3grader.tar) and extract A3grader.py from it.

In [266]:
%run -i A3grader.py


======================= Code Execution =======================

Extracting python code from notebook named 'Lynn-A3.ipynb' and storing in notebookcode.py
Removing all statements that are not function or class defs or import statements.

Testing actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actionsF_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your takeActionsF_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goalTestF_8p correctly True

Testing aStarSearch(1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actionsF_8p, takeActionF_8p,
                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 